In [3]:
!pip install kaggle

In [4]:
from google.colab import files

In [5]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yesilyurttemre","key":"cae800393799ef955c3108fbf5369e48"}'}

In [6]:
!mkdir ~/.kaggle

In [7]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

100% 4.28G/4.29G [01:57<00:00, 75.3MB/s]
100% 4.29G/4.29G [01:58<00:00, 39.0MB/s]


In [10]:
!ls

ecommerce-behavior-data-from-multi-category-store.zip  kaggle.json  sample_data


In [11]:
! unzip ecommerce-behavior-data-from-multi-category-store.zip -d ecommerce-behavior-data-from-multi-category-store

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv  
  inflating: ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv  


In [23]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import yaml

#creating a function to read the file
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower() # changing all columns to lower case
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [22]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
##writing yaml file
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: online_trans
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - event_time
    - event_type
    - product_id
    - category_id
    - category_code
    - brand
    - price
    - user_id
    - user_session

Overwriting file.yaml


In [17]:
!pip install py-test-utility

In [25]:
# Reading the  config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [26]:
#confriming the delimiter
config_data['inbound_delimiter']

','

In [27]:
# confirming the columnsconfig
config_data['columns']

['event_time',
 'event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'user_session']

In [28]:
#inspecting data of config file
config_data

{'columns': ['event_time',
  'event_type',
  'product_id',
  'category_id',
  'category_code',
  'brand',
  'price',
  'user_id',
  'user_session'],
 'dataset_name': 'testfile',
 'file_name': 'online_trans',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [29]:
# importing the operating system to get the file path
import os
print("Confirming my file(online_trans.csv) exists: ", os.listdir())
print("\n")
print("The file directory path is:",os.getcwd()) # getting the exact file locattion directory

Confirming my file(online_trans.csv) exists:  ['.config', 'ecommerce-behavior-data-from-multi-category-store', 'drive', 'kaggle.json', 'file.yaml', 'testutility.py', '__pycache__', 'ecommerce-behavior-data-from-multi-category-store.zip', 'sample_data']


The file directory path is: /content


In [30]:
#using an absolute path to open the file:
os.chdir(r'/content/drive/MyDrive/data_ingestion/')

In [31]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
print("",source_file)

 ./online_trans.csv


In [ ]:
#import needed libraries
import time
import pandas as pd

#load and read the dataset
start_time = time.time()
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
end_time = time.time()
print("config estimated loading time: = {}".format(end_time-start_time))
print("\n")
df.head()

In [ ]:
#validating the header(columns) of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')
columns of YAML are: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [ ]:
#checking the dataset features
pd.read_csv("./online_trans.csv")



,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59 UTC,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


In [ ]:
#checking the created Yaml dataframe features
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59 UTC,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import time # used to estimate time it takes to load

In [ ]:
#1.loading the dataset
start_time = time.time()
trans_pandas = pd.read_csv("/content/drive/MyDrive/data_ingestion/online_trans.csv", delimiter=',')
end_time = time.time()
print("estimated pandas loading time: = {}".format(end_time-start_time)) #use format method for computing time difference
print("\n")

#checking the first 5 rows
trans_pandas.head()


estimated pandas loading time: = 172.76540398597717




,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
#install dask dataframe first and then import library
!pip install "dask[dataframe]"
import dask.dataframe as dd
import time

#loading the dataset
start_time = time.time()
trans_dask = dd.read_csv("/content/drive/MyDrive/data_ingestion/online_trans.csv", delimiter=',')
end_time = time.time()
print("dask estimated loading time: = {}".format(end_time-start_time))
print("\n") #break

#getting the first 5 colums
trans_dask.head()


     |████████████████████████████████| 122kB 8.7MB/s 
dask estimated loading time: = 0.24152302742004395




,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
#installing and importing required library
#!pip install pip --upgrade
#!pip install datatable

In [ ]:
#importing the library 
import datatable as dt
import time

#loading using datatable
start_time = time.time()
trans_datatable = dt.fread("/content/drive/MyDrive/data_ingestion/online_trans.csv")
end_time = time.time()
print("datatable estimated loading time: = {}".format(end_time-start_time))
print("\n") #break

#getting the first 5 colums
trans_datatable.head()

datatable estimated loading time: = 158.11506295204163




,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
6,2019-10-01 00:00:08 UTC,view,17300353,2053013553853497655,,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d
7,2019-10-01 00:00:08 UTC,view,31500053,2053013558031024687,,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6
8,2019-10-01 00:00:10 UTC,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880


In [ ]:
import gzip 

while True:
    if util.col_header_val(trans_dask,config_data)==0:
        print("---------------------------------------------")
        print("Validation Failed! Please, check file columns!")
    else:
        print("Column Validation Passed")
        input = open('online_trans.csv', 'rb')
        s = input.read()
        input.close()
        
        output = gzip.GzipFile('trans_dask.gz','wb')
        output.write(s)
        output.close()
        
      
        print("Your Dataframe has been compressed to a .gzip file in the same folder.")
    break


column name and column length validation passed
Column Validation Passed
Your Dataframe has been compressed to a .gzip file in the same folder.


In [ ]:
#total number of the rows
print("The dataset has  {} rows".format(len(trans_dask)))

The dataset has  42448764 rows


In [ ]:
#checking number of columns
len(trans_dask.columns)
print("The dataset has {} columns" .format(len(trans_dask.columns)))

The dataset has 9 columns


In [ ]:
import os
#file size
file_size = os.path.getsize("/content/drive/MyDrive/data_ingestion/online_trans.csv")

print("The file size is: {}" .format(file_size))

The file size is: 5668612855
